<a href="https://colab.research.google.com/github/walkerjian/DailyCode/blob/main/Copy_of_Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You are given an N by N matrix of random letters and a dictionary of words. Find the maximum number of words that can be packed on the board from the given dictionary.

A word is considered to be able to be packed on the board if:

It can be found in the dictionary
It can be constructed from untaken letters by other words found so far on the board
The letters are adjacent to each other (vertically and horizontally, not diagonally).
Each tile can be visited only once by any word.

For example, given the following dictionary:

{ 'eat', 'rain', 'in', 'rat' }
and matrix:
```
[['e', 'a', 'n'],
 ['t', 't', 'i'],
 ['a', 'r', 'a']]
 ```
Your function should return 3, since we can make the words 'eat', 'in', and 'rat' without them touching each other. We could have alternatively made 'eat' and 'rain', but that would be incorrect since that's only 2 words.

To solve this problem, we can utilize a backtracking approach along with a trie data structure for efficient word lookup and prefix checking. Here's a step-by-step breakdown of the algorithm:

1. **Trie Construction**: First, we'll build a trie from the dictionary words. This trie will help in efficient lookup of words and prefixes during our search.

2. **Backtracking Search**: We'll use a recursive backtracking approach to explore all possible word paths on the board. For each cell, we'll start a depth-first search (DFS) that tries to form words by exploring adjacent cells (left, right, up, down).

3. **Board State Management**: We'll maintain a dynamic state of the board using a 2D boolean array to mark cells that are currently used by the forming word to ensure each letter is only used once per word.

4. **Word Validation**: As we build potential words using DFS, we check:
   - If the current sequence of letters is a prefix to any word in the dictionary. If not, we prune the search.
   - If the sequence is a complete word in the dictionary, we mark it as found, continue to try to find more words with remaining letters.

5. **Maximization Strategy**: We aim to find the maximum number of words that can be formed. This requires managing and updating the best solution found at each step of the recursion. We'll keep track of the words formed and ensure that when exploring different paths, we maximize the number of non-overlapping words.

6. **Result Compilation**: After exploring all possibilities, we return the maximum count of non-overlapping words that can be formed.


This code sets up the Trie for dictionary lookup, then performs a DFS for each position in the board matrix, attempting to form and count valid words. The approach ensures that each letter can only be used once per found word and maximizes the count of non

In [1]:
class TrieNode:
    def __init__(self):
        self.children = {}
        self.is_word = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
        node.is_word = True

    def starts_with(self, prefix):
        node = self.root
        for char in prefix:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

    def is_word(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return node.is_word

def max_words_on_board(board, words):
    trie = Trie()
    for word in words:
        trie.insert(word)

    n = len(board)
    max_words = 0
    path = []

    def dfs(x, y, node, path, visited):
        if x < 0 or x >= n or y < 0 or y >= n or visited[x][y]:
            return 0

        char = board[x][y]
        if char not in node.children:
            return 0

        visited[x][y] = True
        node = node.children[char]
        path.append(char)

        # Check if current path forms a word
        word_count = 0
        if node.is_word:
            node.is_word = False  # Temporarily mark this word as used
            word_count = 1 + max(
                dfs(i, j, trie.root, [], visited)
                for i in range(n) for j in range(n)
            )
            node.is_word = True  # Restore the word for other paths

        # Continue DFS to form longer words
        word_count = max(word_count,
            dfs(x + 1, y, node, path, visited),
            dfs(x - 1, y, node, path, visited),
            dfs(x, y + 1, node, path, visited),
            dfs(x, y - 1, node, path, visited)
        )

        visited[x][y] = False
        path.pop()

        return word_count

    for i in range(n):
        for j in range(n):
            visited = [[False] * n for _ in range(n)]
            max_words = max(max_words, dfs(i, j, trie.root, [], visited))

    return max_words

# Example usage:
dictionary = { 'eat', 'rain', 'in', 'rat' }
matrix = [
    ['e', 'a', 'n'],
    ['t', 't', 'i'],
    ['a', 'r', 'a']
]
print(max_words_on_board(matrix, dictionary))  # Output should be 3

3
